## BUSQUEDA BOLEANA BASICA EN DOCUMENTOS
- Kevin Maldonado

### Objetivos

Ampliar la funcionalidad de búsqueda de términos simples para incluir capacidades de búsqueda booleana. Esto permitirá a los usuarios realizar consultas más complejas combinando múltiples términos de búsqueda utilizando operadores booleanos.

### Descripción del problema

Debe mejorar el motor de búsqueda existente del ejercicio anterior para que admita operadores booleanos: Y, O y NO. Esto permitirá la recuperación de documentos basándose en las relaciones lógicas entre múltiples términos.

### PASO 1: Preparacion de datos

Asegúrese de que los documentos todavía estén cargados y preprocesados ​​desde la tarea anterior. Los datos deben estar limpios y listos para consultas avanzadas.


### Librerias

In [1]:
import os
import re 


Abre y carga un archivo de texto desde una ruta especificada

In [2]:
def cargar_archivo(ruta_archivo):
    with open(ruta_archivo, "r", encoding="utf-8") as archivo:
        contenido = archivo.read()
    return contenido

Divide el texto en palabras individuales utilizando expresiones regulares

In [3]:
def tokenizar_texto(texto):
    palabras = re.findall(r'\b\d{2,}\b|\b\w+\b', texto)
    return palabras

Convierte todas las palabras a minusculas y eliminar aquellas que no contien caracteres alfabeticos.

In [4]:
def limpiar_texto(palabras):
    palabras_limpias = [palabra.lower() for palabra in palabras if palabra.isalpha()]
    return palabras_limpias


Combina las tres funciones anteriores para cargar un archivo, tokenizar su contenido y limpiar las palabras.

In [5]:
def procesar_archivo(ruta_archivo):
    contenido = cargar_archivo(ruta_archivo)
    palabras_tokenizadas = tokenizar_texto(contenido)
    palabras_limpias = limpiar_texto(palabras_tokenizadas)
    return palabras_limpias


### PASO 2: Preparacion de archivos tokenizados

Recorre todos los archivos en un directorio, procesa cada archivo y almacena las palabras limpias en un diccionario:

    - claves: nombres de los archivos
    - valores: las listas de palabras limpias.

In [6]:
def preparar_archivos_tokenizados(directorio):
    archivos_tokenizados = {}
    for nombre_archivo in os.listdir(directorio):
        if nombre_archivo.endswith(".txt"):
            ruta_archivo = os.path.join(directorio, nombre_archivo)
            palabras_procesadas = procesar_archivo(ruta_archivo)
            archivos_tokenizados[nombre_archivo] = palabras_procesadas
    return archivos_tokenizados

### PASO 3: Creacion de un indice invertido

Se crea un índice invertido a partir de un diccionario de archivos tokenizados.
Recorre cada archivo y su lista de palabras, y construye un diccionario donde las claves son las palabras y los valores son conjuntos de nombres de archivos donde aparece cada palabra.

In [7]:
def crear_indice_invertido(archivos_tokenizados):
    indice_invertido = {}
    for nombre_archivo, palabras in archivos_tokenizados.items():
        for palabra in palabras:
            if palabra in indice_invertido:
                indice_invertido[palabra].add(nombre_archivo)
            else:
                indice_invertido[palabra] = {nombre_archivo}
    return indice_invertido

### PASO 4: Implementación de busqueda  boleana

- Analizar la consulta: implemente una función para analizar la consulta de entrada para identificar los términos y operadores.
- Buscar documentos: según la consulta analizada, implemente la lógica para recuperar y clasificar los documentos según las expresiones booleanas.
- Manejo de distinción entre mayúsculas y minúsculas y coincidencias parciales: opcionalmente, puede manejar casos y coincidencias parciales para refinar los resultados de la búsqueda.

Divide la consulta en términos individuales y separa los operadores booleanos ("AND", "OR", "NOT") en una lista aparte. 

Devuelve dos listas: una lista de términos y una lista de operadores booleanos.

In [8]:
def analizar_consulta(consulta):
    terminos = consulta.split()
    operadores_booleanos = [term for term in terminos if term in ["AND", "OR", "NOT"]]
    terminos = [term for term in terminos if term not in ["AND", "OR", "NOT"]]
    return terminos, operadores_booleanos

En esta funcion, utilizamos:

- los términos de búsqueda
- los operadores booleanos
- El índice invertido 

para encontrar los documentos relevantes que coinciden con la consulta de busqueda. 


In [9]:
def buscar_documentos(terminos, operadores_booleanos, indice_invertido):
    documentos_coincidentes = None
    for i, termino in enumerate(terminos):
        documentos_termino = indice_invertido.get(termino, set())
        if operadores_booleanos and i < len(operadores_booleanos):
            operador = operadores_booleanos[i]
            if operador == "AND":
                documentos_coincidentes = documentos_termino if documentos_coincidentes is None else documentos_coincidentes & documentos_termino
            elif operador == "OR":
                documentos_coincidentes = documentos_termino if documentos_coincidentes is None else documentos_coincidentes | documentos_termino
            elif operador == "NOT":
                documentos_coincidentes -= documentos_termino
        else:
            documentos_coincidentes = documentos_termino if documentos_coincidentes is None else documentos_coincidentes & documentos_termino
    return documentos_coincidentes


### PASO 5: Desplegar los resultados

Esta función muestra los nombres de los documentos que coinciden con la consulta que se ha hecho.

In [10]:
def mostrar_resultados(documentos_coincidentes):
    if documentos_coincidentes:
        print("\nDocumentos que coinciden con la consulta:")
        for documento in documentos_coincidentes:
            print("- ", documento)
    else:
        print("No se encontraron documentos que coincidan con la consulta.")

def obtener_consulta_usuario():
    consulta = input("Ingrese la consulta de búsqueda: ")
    return consulta


Se toma el directorio de archivos de texto a ser tokenizados, para luego construir los indices invertidos(mediante ED diccionario).


In [11]:
directorio = r"D:\SEPTIMO SEMESTRE II\RECUPERACION INFORMACION\KevinMaldonado99\RETRIEVAL INFO\week01\TASK 2\Data"
archivos_tokenizados = preparar_archivos_tokenizados(directorio)
indice_invertido = crear_indice_invertido(archivos_tokenizados)


Solicita y analiza la consulta para identificar el termino de busqueda utilizando el indice invertido para encontrar los docuemntos que coinciden con la consulta.

In [12]:
consulta_usuario = obtener_consulta_usuario()
terminos, operadores_booleanos = analizar_consulta(consulta_usuario)
documentos_coincidentes = buscar_documentos(terminos, operadores_booleanos, indice_invertido)
mostrar_resultados(documentos_coincidentes)


Ingrese la consulta de búsqueda: the AND book AND NOT enter

Documentos que coinciden con la consulta:
-  13.txt
-  16.txt
-  77.txt
-  78.txt
-  86.txt
-  98.txt


Genera una matriz que representa las palabras en los archivos tokenizados. 

Cada fila de la matriz corresponde a una palabra, y cada columna corresponde a un archivo. Si una palabra está presente en un archivo, se marca con un 1 en la matriz; de lo contrario, se marca con un 0. 

In [13]:
def crear_matriz_indices(archivos_tokenizados):
    matriz_indices = {}
    for nombre_archivo, palabras in archivos_tokenizados.items():
        for palabra in palabras:
            if palabra in matriz_indices:
                matriz_indices[palabra][nombre_archivo] = 1
            else:
                matriz_indices[palabra] = {archivo: 0 for archivo in archivos_tokenizados}
                matriz_indices[palabra][nombre_archivo] = 1
    return matriz_indices

Muestra las palabras en las filas y los nombres de los archivos en las columnas. Cada celda de la matriz contiene un 1 si la palabra está presente en el archivo correspondiente, y un 0 si no lo está. 

In [14]:
def mostrar_matriz_indices(matriz_indices):
    print("\nMatriz de índices:")
    print("{: <15}".format("Palabra"), end="")
    for nombre_archivo in matriz_indices[next(iter(matriz_indices))]:
        print("{: <10}".format(nombre_archivo), end="")
    print()
    for palabra, presencias in matriz_indices.items():
        print("{: <15}".format(palabra), end="")
        for nombre_archivo in matriz_indices[next(iter(matriz_indices))]:
            print("{: <10}".format(presencias.get(nombre_archivo, 0)), end="")
        print()

### PASO 6: Matriz de indices

Imprime la matriz de índices en la consola. 

In [15]:
# Luego de obtener la consulta y los documentos coincidentes

# Preparar la matriz de índices
matriz_indices = crear_matriz_indices(archivos_tokenizados)

# Mostrar la matriz de índices
mostrar_matriz_indices(matriz_indices)



Matriz de índices:
Palabra        100.txt   13.txt    16.txt    17.txt    77.txt    78.txt    79.txt    86.txt    98.txt    99.txt    
the            1         1         1         1         1         1         1         1         1         1         
project        1         1         1         1         1         1         1         1         1         1         
gutenberg      1         1         1         1         1         1         1         1         1         1         
ebook          1         1         1         1         1         1         1         1         1         1         
of             1         1         1         1         1         1         1         1         1         1         
adventures     1         1         0         0         0         1         1         1         0         0         
roderick       1         0         0         0         0         0         0         0         0         0         
random         1         0         1         0      